In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from bin import *

from IPython.display import clear_output
%load_ext autoreload
%autoreload 2

In [2]:
mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'font.size' : 8,
    'text.usetex': True,
    'pgf.rcfonts': False,
    'pgf.preamble': tex_preamble,
})

In [3]:
n = 50
Λ = model_problem_spectrum(n,.6,1e3)
λmin = np.min(Λ)
λmax = np.max(Λ)

A = np.diag(Λ)

b = np.ones(n)
b /= np.linalg.norm(b)

f = lambda x: 1/x

fAb = np.diag(f(Λ))@b

In [4]:
k_max = 23
Q_ro,(α_ro,β_ro) = lanczos_reorth(A,b,k_max,reorth=k_max)

err_FA_ro = np.full(k_max,np.nan)
for k in range(k_max):
    err_FA_ro[k] = np.linalg.norm(np.diag(np.sqrt(Λ))@(fAb - lanczos_FA(f,Q_ro,α_ro,β_ro,k)))/np.linalg.norm(np.diag(np.sqrt(Λ))@b)

In [5]:
k_max = 44
Q,(α,β) = lanczos_reorth(A,b,k_max,reorth=0)

err_FA = np.full(k_max,np.nan)
for k in range(k_max):
    err_FA[k] = np.linalg.norm(np.diag(np.sqrt(Λ))@(fAb - lanczos_FA(f,Q,α,β,k)))/np.linalg.norm(np.diag(np.sqrt(Λ))@b)

In [6]:
k_max = 44
cluster_size = 10
Λ_sim = (Λ[:,None] + np.linspace(-6e-14,6e-14,cluster_size)).flatten()
A_sim = np.diag(Λ_sim)
b_sim = np.ones(n*cluster_size)
b_sim /= np.linalg.norm(b_sim)

fAb_sim = np.diag(f(Λ_sim))@b_sim

Q_sim,(α_sim_ro,β_sim_ro) = lanczos_reorth(A_sim,b_sim,k_max,reorth=k)
Q_sim,(α_sim,β_sim) = lanczos_reorth(A_sim,b_sim,k_max,reorth=0)

err_FA_sim = np.full(k_max,np.nan)
for k in range(k_max):
    err_FA_sim[k] = np.linalg.norm(np.diag(np.sqrt(Λ_sim))@(fAb_sim - lanczos_FA(f,Q_sim,α_sim,β_sim,k)))/np.linalg.norm(np.diag(np.sqrt(Λ_sim))@b_sim)

In [7]:
fig = plt.figure(figsize=(figure_width*cm,figure_height*cm))

ax1 = fig.add_axes([left, bottom+height/2, width, height/2.2])
ax2 = fig.add_axes([left, bottom, width, height/2.2])

ax1.plot(α,**line_styles['l1'])
ax1.plot(α_ro,**line_styles['l3'])
ax1.plot(α_sim_ro,**line_styles['l2'])
# ax1.plot(α_sim,**line_styles['l4'])

ax2.plot(β,**line_styles['l1'])
ax2.plot(β_ro,**line_styles['l3'])
ax2.plot(β_sim_ro,**line_styles['l2'])
# ax2.plot(β_sim,**line_styles['l4'])
ax2.set_xlabel('index: $k$')
ax1.set_ylabel('error')
ax2.set_ylabel('error')

ax1.set_yscale('log')
ax2.set_yscale('log')

plt.savefig('imgs/greeenbaum_intervals.pdf')
plt.close()

In [8]:
fig = plt.figure(figsize=(figure_width*cm,figure_height*cm))

ax = fig.add_axes([left, bottom, width, height])

ax.plot(err_FA,**line_styles['l1'],label='no reorth')
ax.plot(err_FA_sim,**line_styles['l3'],label='clusters')
ax.plot(err_FA_ro,**line_styles['l2'],label='reorth')

ax.grid(True,linestyle=':',linewidth=.5)

ax.set_xlabel('number of matvecs: $k$')
ax.set_ylabel('error norm')
ax.set_yscale('log')

plt.savefig('imgs/greeenbaum_convergence.pdf')
plt.close()